
# Documentação do uso do Flair para dataset de Substâncias

Este notebook utiliza a biblioteca `Flair` para executar uma tarefa de classificação de texto com dados pré-processados e estruturados. Abaixo está a descrição detalhada da estrutura e do propósito de cada etapa.

---

## Estrutura Geral do Notebook

1. **Instalação de Dependências**: Garantindo que os pacotes necessários estão disponíveis.
2. **Importação de Bibliotecas**: Inclui tanto bibliotecas do Python padrão quanto específicas, como o `Flair` e ferramentas auxiliares como o `sklearn`.
3. **Integração com Google Drive**: Carregando arquivos armazenados no Google Drive para o ambiente de execução.
4. **Preparação de Dados**: Carregamento, limpeza e divisão de dados em conjuntos de treino, validação e teste.
5. **Configuração do Modelo**: Inicialização e treinamento usando embeddings baseados em modelos de linguagem (transformadores).
6. **Treinamento e Validação**: Executando o fine-tuning e salvando o modelo treinado.

---

## Etapas do Notebook

### 1. Instalação de Dependências
O pacote `Flair` é instalado para manipulação de tarefas de processamento de linguagem natural (PNL).

```python
!pip install flair
```

---

### 2. Importação de Bibliotecas
As bibliotecas principais utilizadas incluem:

- **Pandas e NumPy**: Manipulação e análise de dados.
- **Flair**: Classificação de texto e embeddings de transformadores.
- **sklearn**: Divisão dos dados em treino, validação e teste.

```python
import csv, os
import pandas as pd
import numpy as np
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from sklearn.model_selection import train_test_split
```

---

### 3. Preparação de Dados
Os datasets de treino, validação e teste são carregados diretamente do GitHub.

```python
train_file_path = "https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_train.csv"
val_file_path = 'https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_val.csv'
test_file_path = 'https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_test.csv'

train_file = pd.read_csv(train_file_path)
val_file = pd.read_csv(val_file_path)
test_file = pd.read_csv(test_file_path)
```

---

### 4. Configuração do Modelo
Os embeddings são configurados usando o modelo BERT em português da Neuralmind. Um corpus é criado para organizar os dados no formato esperado pelo `Flair`.

```python
column_name_map = {0: "text", 1: "label"}
corpus = CSVClassificationCorpus(
    data_folder="",
    column_name_map=column_name_map,
    train_file=train_file_path_all,
    test_file=test_file_path,
    dev_file=val_file_path,
    label_type='substancia'
)

document_embeddings = TransformerDocumentEmbeddings(
    'neuralmind/bert-base-portuguese-cased', fine_tune=True
)

classifier = TextClassifier(
    document_embeddings, label_dictionary=corpus.make_label_dictionary(label_type='substancia')
)
```

---

### 5. Treinamento e Validação
O treinamento do modelo é realizado com fine-tuning e os resultados são salvos no ambiente de execução.

```python
trainer = ModelTrainer(classifier, corpus)
trainer.train('/substances-classification-with-flair',
              learning_rate=0.1,
              mini_batch_size=64,
              patience=5,
              max_epochs=200,
              main_evaluation_metric=("macro avg", 'f1-score'))
```

---

## Análise Geral da Arquitetura

### Uso de Datasets
- **Treino**: Para ensinar o modelo.
- **Validação**: Para ajustar o modelo durante o treinamento.
- **Teste**: Para avaliar a performance final em dados não vistos.

### Modelo Utilizado
- **BERT-base-portuguese-cased**: Modelo pré-treinado em português.

### Pipeline de Treinamento
1. Dados estruturados no formato do `Flair`.
2. Fine-tuning de embeddings pré-treinados para a tarefa específica.
3. Salvar resultados e modelos no Google Drive.

---

In [2]:
!pip install flair

In [3]:
# Imports
import csv, os
import pandas as pd
import numpy as np
from flair.data import Corpus
from flair.datasets import CSVClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

from sklearn.model_selection import train_test_split

In [4]:
# Get datasets
train_file_path = "https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_train.csv"
val_file_path = "https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_val.csv"
test_file_path = "https://raw.githubusercontent.com/noharm-ai/substance-prediction/refs/heads/main/datasets/med_all_bio_test.csv"

train_file = pd.read_csv(train_file_path)
val_file = pd.read_csv(val_file_path)
test_file = pd.read_csv(test_file_path)

## Preprocessing

In [5]:
# Therefore we define a mininum number of examples per class
min_samples_per_class = 10

# Find classes to remove
full = pd.concat([train_file, test_file, val_file])
class_counts = full['substancia'].value_counts()
classes_to_remove = class_counts[class_counts < min_samples_per_class].index

# Filter all dataframes
filtered_dataset = full[~full['substancia'].isin(classes_to_remove)]
train_filtered = train_file[~train_file['substancia'].isin(classes_to_remove)]
test_filtered = test_file[~test_file['substancia'].isin(classes_to_remove)]
val_filtered = val_file[~val_file['substancia'].isin(classes_to_remove)]

In [6]:
train_data_2, test_data_2 = train_test_split(val_filtered, test_size=0.067, random_state=42, shuffle=True)

train_file_path_all = '/bio_train_all.csv'
train_filtered_path = '/bio_train_filtered.csv'
val_filtered_path = '/bio_val_filtered.csv'
test_filtered_path = '/bio_test_filtered.csv'

train_file_full = pd.concat([train_file, train_data_2], ignore_index=True)

train_file_full.to_csv(train_file_path_all, index=False)
val_filtered.to_csv(val_filtered_path, index=False)
test_data_2.to_csv(test_filtered_path, index=False)

In [ ]:
train_file_full.describe()

,medicamento,substancia
count,78292,78292
unique,68684,2094
top,"CLONAZEPAM 2,5MG/ML","CLORETO DE SODIO 0,9% (SORO FISIOLOGICO)"
freq,40,1117


In [ ]:
# 1. Preparar o Corpus do Flair
# Colunas: texto (medicamentos) e rótulo (substancias)
column_name_map = {0: "text", 1: "label"}  # Mapear as colunas com base no índice

corpus = CSVClassificationCorpus(
    data_folder="",
    column_name_map=column_name_map,
    train_file=train_file_path_all,
    test_file=test_filtered_path,
    dev_file=val_filtered_path,
    label_type='substancia',  # Tipo de label definido
    delimiter=','
)

# 2. Definir o tipo de rótulo
label_type = 'substancia'  # Ajuste o nome do tipo de rótulo se necessário

# 3. Criar o dicionário de rótulos
label_dict = corpus.make_label_dictionary(label_type=label_type)

# 4. Inicializar as embeddings do transformador usando o modelo BERT em português
document_embeddings = TransformerDocumentEmbeddings('neuralmind/bert-base-portuguese-cased', fine_tune=True)

# 5. Criar o classificador de texto
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type=label_type)

# 6. Inicializar o treinador
trainer = ModelTrainer(classifier, corpus)

# 7. Executar o treinamento com fine-tuning
trainer.train('/substances-classification-with-flair',
                  learning_rate=0.1,
                  mini_batch_size=64,
                  patience=5,
                  max_epochs=200,
                  main_evaluation_metric=("macro avg", 'f1-score'))

2024-11-26 21:03:07,704 Reading data from .
2024-11-26 21:03:07,707 Train: /bio_train_all.csv
2024-11-26 21:03:07,709 Dev: /bio_val_filtered.csv
2024-11-26 21:03:07,710 Test: /bio_test_filtered.csv
2024-11-26 21:03:08,268 Computing label dictionary. Progress:


0it [00:00, ?it/s]
78293it [00:17, 4586.52it/s]

2024-11-26 21:03:25,374 Dictionary created for label 'substancia' with 2095 values: CLORETO DE SODIO 0,9% (SORO FISIOLOGICO) (seen 1117 times), SOLUCAO DE GLICOSE (SORO GLICOSADO) (seen 755 times), SULFATO DE MORFINA (seen 566 times), LevoTIROXina (seen 559 times), MIDAZOLAM (seen 547 times), DipiRONA (seen 487 times), ENOXAPARINA SODICA (seen 418 times), DEXAMETASONA (seen 417 times), FENTANILA (seen 391 times), HaloPERidol (seen 390 times), LIDOCAINA CLORIDRATO (seen 383 times), ONDANSETRONA (seen 382 times), CLONAZEPAM (seen 368 times), PARACETAMOL  (ACETAMINOFEM) (seen 366 times), ClorproMAZINA (seen 350 times), FENOBARBITAL (seen 347 times), ISOSSORBIDA (seen 346 times), FUROSEMIDA (seen 344 times), TRAMADOL CLORIDRATO (seen 339 times), SULFAMETOXAZOL+TRIMETOPRIMA (seen 338 times)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


2024-11-26 21:03:30,107 ----------------------------------------------------------------------------------------------------
2024-11-26 21:03:30,110 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29795, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
            

...

2024-11-27 00:29:42,566 DEV : loss 0.028607947751879692 - f1-score (macro avg)  0.9839
2024-11-27 00:29:45,680  - 4 epochs without improvement
2024-11-27 00:29:45,682 ----------------------------------------------------------------------------------------------------
2024-11-27 00:30:11,475 epoch 45 - iter 122/1224 - loss 0.01589486 - time (sec): 25.79 - samples/sec: 302.76 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:30:35,830 epoch 45 - iter 244/1224 - loss 0.01678695 - time (sec): 50.14 - samples/sec: 311.42 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:31:00,130 epoch 45 - iter 366/1224 - loss 0.01829564 - time (sec): 74.44 - samples/sec: 314.65 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:31:26,078 epoch 45 - iter 488/1224 - loss 0.01713850 - time (sec): 100.39 - samples/sec: 311.10 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:31:50,938 epoch 45 - iter 610/1224 - loss 0.01750007 - time (sec): 125.25 - samples/sec: 311.69 - lr: 0.050000 - momentum: 0.000000
2024-

100%|██████████| 227/227 [00:24<00:00,  9.19it/s]


2024-11-27 00:34:22,789 DEV : loss 0.02837086096405983 - f1-score (macro avg)  0.9804
2024-11-27 00:34:25,858  - 5 epochs without improvement
2024-11-27 00:34:25,860 ----------------------------------------------------------------------------------------------------
2024-11-27 00:34:51,534 epoch 46 - iter 122/1224 - loss 0.01121987 - time (sec): 25.67 - samples/sec: 304.16 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:35:17,262 epoch 46 - iter 244/1224 - loss 0.01516709 - time (sec): 51.40 - samples/sec: 303.82 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:35:42,688 epoch 46 - iter 366/1224 - loss 0.01721618 - time (sec): 76.82 - samples/sec: 304.90 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:36:07,912 epoch 46 - iter 488/1224 - loss 0.01601048 - time (sec): 102.05 - samples/sec: 306.05 - lr: 0.050000 - momentum: 0.000000
2024-11-27 00:36:33,042 epoch 46 - iter 610/1224 - loss 0.01633012 - time (sec): 127.18 - samples/sec: 306.97 - lr: 0.050000 - momentum: 0.000000
2024-1

100%|██████████| 227/227 [00:24<00:00,  9.19it/s]


2024-11-27 00:39:02,091 DEV : loss 0.025531530380249023 - f1-score (macro avg)  0.9879
2024-11-27 00:39:05,173  - 6 epochs without improvement (above 'patience')-> annealing learning_rate to [0.025]
2024-11-27 00:39:05,176 ----------------------------------------------------------------------------------------------------
2024-11-27 00:39:28,958 epoch 47 - iter 122/1224 - loss 0.01727304 - time (sec): 23.78 - samples/sec: 328.37 - lr: 0.025000 - momentum: 0.000000
2024-11-27 00:39:54,064 epoch 47 - iter 244/1224 - loss 0.01350405 - time (sec): 48.88 - samples/sec: 319.45 - lr: 0.025000 - momentum: 0.000000
2024-11-27 00:40:19,113 epoch 47 - iter 366/1224 - loss 0.01251924 - time (sec): 73.93 - samples/sec: 316.82 - lr: 0.025000 - momentum: 0.000000
2024-11-27 00:40:44,607 epoch 47 - iter 488/1224 - loss 0.01399398 - time (sec): 99.43 - samples/sec: 314.12 - lr: 0.025000 - momentum: 0.000000
2024-11-27 00:41:09,596 epoch 47 - iter 610/1224 - loss 0.01294635 - time (sec): 124.42 - sample


## 7. Avaliação do Modelo

Após o treinamento, utilizamos o conjunto de teste para calcular métricas de desempenho, incluindo **precisão**, **recall** e **F1-score**. Essas métricas ajudam a avaliar a qualidade do modelo em dados não vistos.

As etapas para avaliação incluem:
1. Carregar o modelo treinado.
2. Prever os rótulos para o conjunto de teste.
3. Calcular as métricas usando `sklearn`.

---


In [10]:
from flair.models import TextClassifier
from flair.data import Sentence
from sklearn.metrics import classification_report
import pandas as pd

# Carregar o modelo treinado
model_path = '/substances-classification-with-flair/final-model.pt'
classifier = TextClassifier.load(model_path)

# Carregar os dados de teste
test_data = pd.read_csv(test_filtered_path)

# Preparar os dados no formato esperado pelo Flair
test_sentences = []
for row in test_data.values:
    sentence = Sentence(row[1])  # O texto é passado diretamente
    sentence.add_label("substancia", row[0])  # Adiciona o rótulo associado
    test_sentences.append(sentence)

# Fazer previsões com o modelo no conjunto de teste
true_labels = []
predicted_labels = []

for sentence in test_sentences:
    classifier.predict(sentence)
    true_labels.append(sentence.labels[0].value)  # Rótulos reais
    predicted_labels.append(sentence.labels[0].value)  # Rótulos previstos pelo modelo

# Gerar o relatório com métricas usando sklearn
report = classification_report(true_labels, predicted_labels)
print(report)

                                                                                          precision    recall  f1-score   support

(RINGER LACTATO) CLORETO DE SODIO+CLORETO DE POTASSIO+CLORETO DE CALCIO+LACTATO DE SODIO       1.00      1.00      1.00         3
                                                                                ABACAVIR       1.00      1.00      1.00         1
                                                                             ABIRATERONA       1.00      1.00      1.00         1
                                                      ACETATO DE HIDROCORTISONA (TOPICO)       1.00      1.00      1.00         1
                                                                           ACETAZOLAMIDA       1.00      1.00      1.00         1
                                                                          ACETILCISTEINA       1.00      1.00      1.00         2
                                                                               ACICLOVIR 